In [1]:
import gurobipy as gp
from gurobipy import GRB

env = gp.Env(empty=True)
env.setParam("OutputFlag",0)
env.start()

<gurobipy.Env, Parameter changes: OutputFlag=0>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [63]:
# read files
# bus, generator, branch
df_bus = pd.read_csv("bus.csv", header=None)
df_generator = pd.read_csv("case5_gen.csv",header=None)
df_branch = pd.read_csv("case5_branch.csv",header=None)

In [64]:
df_bus.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,2,0,0.00,0,0,1,1,0,230,1,1.1,0.9
1,2,1,300,98.61,0,0,1,1,0,230,1,1.1,0.9
2,3,2,300,98.61,0,0,1,1,0,230,1,1.1,0.9
3,4,3,400,131.47,0,0,1,1,0,230,1,1.1,0.9
4,5,2,0,0.00,0,0,1,1,0,230,1,1.1,0.9


In [65]:
df_generator.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,1,40.00,0,30.0,-30.0,1,100,1,40,0,...,0,0,0,0,0,0,0,0,0,0
1,1,170.00,0,127.5,-127.5,1,100,1,170,0,...,0,0,0,0,0,0,0,0,0,0
2,3,323.49,0,390.0,-390.0,1,100,1,520,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0.00,0,150.0,-150.0,1,100,1,200,0,...,0,0,0,0,0,0,0,0,0,0
4,5,466.51,0,450.0,-450.0,1,100,1,600,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
df_branch.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,2,0.00281,0.0281,0.00712,400,400,400,0,0,1,-360,360
1,1,4,0.00304,0.0304,0.00658,0,0,0,0,0,1,-360,360
2,1,5,0.00064,0.0064,0.03126,0,0,0,0,0,1,-360,360
3,2,3,0.00108,0.0108,0.01852,0,0,0,0,0,1,-360,360
4,3,4,0.00297,0.0297,0.00674,0,0,0,0,0,1,-360,360


In [67]:
df_bus = df_bus.drop(3, axis=1)
df_bus = df_bus.drop(4, axis=1)
df_bus = df_bus.drop(5, axis=1)
df_bus = df_bus.drop(7, axis=1)
df_bus = df_bus.drop(8, axis=1)
df_bus = df_bus.drop(9, axis=1)
df_bus = df_bus.drop(10, axis=1)
df_bus = df_bus.rename(
    columns={0: "bus_i", 1: "type", 2: "Pd", 6: "area", 11: "Vmax", 12: "Vmin"}
)
df_bus.head()

,bus_i,type,Pd,area,Vmax,Vmin
0,1,2,0,1,1.1,0.9
1,2,1,300,1,1.1,0.9
2,3,2,300,1,1.1,0.9
3,4,3,400,1,1.1,0.9
4,5,2,0,1,1.1,0.9


In [57]:
df_generator = df_generator.drop(2, axis=1)
df_generator = df_generator.drop(3, axis=1)
df_generator = df_generator.drop(4, axis=1)
df_generator = df_generator.drop(6, axis=1)
df_generator = df_generator.drop(7, axis=1)
for i in range(10, 21):
    df_generator = df_generator.drop(i, axis=1)
df_generator = df_generator.rename(
    columns={0: "bus", 1: "Pg", 5: "Vg", 8: "Pmax", 9: "Pmin"}
)
df_generator.head()

,bus,Pg,Vg,Pmax,Pmin
0,1,40.00,1,40,0
1,1,170.00,1,170,0
2,3,323.49,1,520,0
3,4,0.00,1,200,0
4,5,466.51,1,600,0


In [59]:
df_generator.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   bus     5 non-null      int64  
 1   Pg      5 non-null      float64
 2   Vg      5 non-null      int64  
 3   Pmax    5 non-null      int64  
 4   Pmin    5 non-null      int64  
dtypes: float64(1), int64(4)
memory usage: 332.0 bytes


In [47]:
for i in range(2, 9):
    df_branch = df_branch.drop(i, axis=1)
df_branch = df_branch.drop(10, axis=1)
df_branch = df_branch.rename(
    columns={0: "fbus", 1: "tbus", 9: "angle", 11: "angmin", 12: "angmax"}
)
df_branch.head()

,fbus,tbus,angle,angmin,angmax
0,1,2,0,-360,360
1,1,4,0,-360,360
2,1,5,0,-360,360
3,2,3,0,-360,360
4,3,4,0,-360,360


In [54]:
L = []
for index,row in df_branch.iterrows():
    L.append((row["fbus"],row["tbus"]))
print(L)

[(1, 2), (1, 4), (1, 5), (2, 3), (3, 4), (4, 5)]


In [34]:
A_e = range(1,1+df_bus["area"].nunique())

range(1, 2)

In [69]:
B=range(1,1+df_bus["bus_i"].nunique())
print(B)

range(1, 6)


In [60]:
G = []
for index, row in df_generator.iterrows():
    G.append((index+1,int(row["bus"])))
print(G)

[(1, 1), (2, 1), (3, 3), (4, 4), (5, 5)]


In [2]:
#################################################
####                   Sets                  ####
#################################################
# set of subareas
# A_e = {"Area1", "Area2"}

# set of days in a week
W = range(7)
# set of hours in a day
H = range(24)

# Loads
D_wh = {"Load1": 100, "Load2": 150}
# Generators
# G = {"Gen1", "Gen2"}
# Transmission lines
# L = {"Line1", "Line2"}
# Buses
#B = {"Bus1", "Bus2"}
# electrical components, union of G,L,B and D_wh
E = G.union(L, B, D_wh)



#################################################
####              Parameters                 ####
#################################################

alpha = 0.5  # Trade-off parameter

# Base risk of wildfire ignition in each area
R_j = {"Area1": 1.2, "Area2": 1.5}

# Relative risk factor for electrical components in areas at each hour
kappa_ejh = {("Gen1", "Area1", 1): 1.1, ("Line1", "Area2", 1): 1.2}

# Meteorological factor at each hour
gamma_h = {1: 1.2, 2: 1.1}

# Risk calculations
R_ejh = {
    (e, j, h): kappa_ejh.get((e, j, h), 1.0) * gamma_h.get(h, 1.0) * R_j[j]  #
    for e in E
    for j in A_e
    for h in H
}
# The .get() method is used to retrieve the value from the dictionary kappa_ejh for the key (e, j, h).
# If the key doesn’t exist, it returns 1.0 as a default value.

R_eh = {
    (e, h): sum(R_ejh[e, j, h] for j in A_e if (e, j, h) in R_ejh) for e in E for h in H
}

# w_dh represents the weighting factor for each load and hour, initializing to 1
w_dh = {(d, h): 1.0 for d in D_wh for h in H}

# Typical load served during standard operational phases for each load and hour
D_dh = {(d, h): D_wh[d] for d in D_wh for h in H}

# Define generator min/max power output
P_min, P_max = 10, 100  # Example values

T_l = {l: 50 for l in L}  # Example thermal limits
b_l = {l: 0.01 for l in L}  # Example susceptance values
theta_max, theta_min = 30, -30  # Example angle limits



In [7]:
def hour_based_wildfire_model(h, alpha, D_wh, A_e, G, L, B, H, P_min,P_max,T_l,b_l,theta_max,theta_min):
    model = gp.Model("Power System and Wildfire Risk Management", env=env)

    #################################################
    ####           Decision Variables            ####
    #################################################

    # indicate whether load l will remain energized during hour h
    z_lh = model.addVars(L, H, vtype=GRB.BINARY, name="z_lh")

    # indicate whether bus i will remain energized during hour h
    z_ih = model.addVars(B, H, vtype=GRB.BINARY, name="z_ih")

    # indicate whether generator l will remain energized during hour h
    z_gh = model.addVars(G, H, vtype=GRB.BINARY, name="z_gh")

    # represent the fraction of the load that that is de-energized during hour h
    x_dh = model.addVars(D_wh, H, vtype=GRB.CONTINUOUS, name="x_dh", lb=0, ub=1)

    # represent the power generation of generators in hour h
    P_gh = model.addVars(G, H, lb=0, name="P_gh")

    # represent the power flows on transmission lines l in L from bus j to bus i in hour h
    P_l_ij_h = model.addVars(L, B, B, H, lb=-GRB.INFINITY, name="P_l_ij_h")

    # represent the voltage angles.
    theta_ih = model.addVars(B, H, lb=-180, ub=180, name="theta_ih")

    # Total wildfire risk calculation
    R_Fire_h = {
        h: sum(x_dh[e, h] * R_ejh.get((e, j, h), 0) for e in D_wh for j in A_e)
        + sum(z_gh[e, h] * R_ejh.get((e, j, h), 0) for e in G for j in A_e)
        + sum(z_lh[e, h] * R_ejh.get((e, j, h), 0) for e in L for j in A_e)
        + sum(z_ih[e, h] * R_ejh.get((e, j, h), 0) for e in B for j in A_e)
        for h in H
    }
    # Calculate the total load successfully distributed across the network within hour h
    D_Tot_h = {h: sum(x_dh[d, h] * w_dh[d, h] * D_dh[d, h] for d in D_wh) for h in H}

    #################################################
    ####           Objective Function            ####
    #################################################

    model.setObjective(
        alpha * D_Tot_h[h] - (1 - alpha) * R_Fire_h[h],
        GRB.MAXIMIZE,
    )
    #################################################
    ####               Constraints               ####
    #################################################

    # Energization constraints
    for i in B:
        # for h in H:
        # Loads must be less than or equal to bus energization status
        model.addConstrs(
            (x_dh[d, h] <= z_ih[i, h] for d in D_wh if i in B),
            name=f"load_energization_{i}_{h}",
        )
        # Generator status must be less than or equal to bus energization status
        model.addConstrs(
            (z_gh[g, h] <= z_ih[i, h] for g in G if i in B),
            name=f"gen_energization_{i}_{h}",
        )
        # Line energization
        model.addConstrs(
            (z_lh[l, h] <= z_ih[i, h] for l in L if i in B),
            name=f"line_energization_{i}_{h}",
        )

    # Generation constraints
    for g in G:
        model.addConstr(P_min * z_gh[g, h] <= P_gh[g, h], name=f"Pmin_{g}_{h}")
        model.addConstr(P_gh[g, h] <= P_max * z_gh[g, h], name=f"Pmax_{g}_{h}")

    # Power flow and node balance constraints
    for i in B:
        model.addConstr(
            sum(P_gh[g, h] for g in G if i in B)
            + sum(P_l_ij_h[l, i, j, h] for l in L for j in B if i in B)
            - sum(x_dh[d, h] * D_wh[d] for d in D_wh if i in B)
            == 0,
            name=f"node_balance_{i}_{h}",
        )

        for l in L:
            for j in B:
                if (
                    i != j
                ):  # Only add flow constraints for lines between different buses
                    model.addConstr(
                        -T_l[l] * z_lh[l, h] <= P_l_ij_h[l, i, j, h],
                        name=f"flow_limit_lower_{l}_{i}_{j}_{h}",
                    )
                    model.addConstr(
                        P_l_ij_h[l, i, j, h] <= T_l[l] * z_lh[l, h],
                        name=f"flow_limit_upper_{l}_{i}_{j}_{h}",
                    )
                    model.addConstr(
                        P_l_ij_h[l, i, j, h]
                        <= -b_l[l]
                        * (
                            theta_ih[i, h]
                            - theta_ih[j, h]
                            + theta_max * (1 - z_lh[l, h])
                        ),
                        name=f"angle_limit_upper_{l}_{i}_{j}_{h}",
                    )
                    model.addConstr(
                        P_l_ij_h[l, i, j, h]
                        >= -b_l[l]
                        * (
                            theta_ih[i, h]
                            - theta_ih[j, h]
                            + theta_min * (1 - z_lh[l, h])
                        ),
                        name=f"angle_limit_lower_{l}_{i}_{j}_{h}",
                    )

    # Solve the model
    model.optimize()

    # print result
    # Check if the optimal solution is found
    if model.status == GRB.OPTIMAL:
        print("Optimal Solution:")

        # Print operational status for generators, lines, and buses
        # for h in H:
        print(f"Hour {h}:")
        for g in G:
            print(
                f"  Generator {g}: Energized = {'Yes' if z_gh[g, h].X > 0.5 else 'No'}, Power Generated = {P_gh[g, h].X:.2f} MW"
            )
        for l in L:
            print(f"  Line {l}: Energized = {'Yes' if z_lh[l, h].X > 0.5 else 'No'}")
        for b in B:
            print(f"  Bus {b}: Energized = {'Yes' if z_ih[b, h].X > 0.5 else 'No'}")

        # Print load shedding details
        for d in D_wh:
            print(f"  Load {d} Shedding: {x_dh[d, h].X * 100:.2f}% of {D_wh[d]} MW")

        # Print power flows on transmission lines if needed
        for l in L:
            for i in B:
                for j in B:
                    if i != j and P_l_ij_h[l, i, j, h].X != 0:
                        print(
                            f"  Power Flow on Line {l} from Bus {i} to Bus {j}: {P_l_ij_h[l, i, j, h].X:.2f} MW"
                        )

        # Print voltage angles at buses if needed
        for b in B:
            print(f"  Voltage Angle at Bus {b}: {theta_ih[b, h].X:.2f} degrees")
        print(f"\n")
    else:
        print("No optimal solution found.")

    

In [8]:
for h in H:
    hour_based_wildfire_model(
        h,
        alpha=alpha,
        D_wh=D_wh,
        A_e=A_e,
        G=G,
        L=L,
        B=B,
        H=H,
        P_min=P_min,
        P_max=P_max,
        T_l=T_l,
        b_l=b_l,
        theta_max=theta_max,
        theta_min=theta_min,
    )
    

Optimal Solution:
Hour 0:
  Generator Gen1: Energized = No, Power Generated = 0.00 MW
  Generator Gen2: Energized = No, Power Generated = 0.00 MW
  Line Line2: Energized = Yes
  Line Line1: Energized = Yes
  Bus Bus2: Energized = Yes
  Bus Bus1: Energized = Yes
  Load Load1 Shedding: 100.00% of 100 MW
  Load Load2 Shedding: 100.00% of 150 MW
  Power Flow on Line Line2 from Bus Bus2 to Bus Bus1: 3.60 MW
  Power Flow on Line Line2 from Bus Bus1 to Bus Bus2: -3.60 MW
  Power Flow on Line Line1 from Bus Bus2 to Bus Bus1: 3.60 MW
  Power Flow on Line Line1 from Bus Bus1 to Bus Bus2: -3.60 MW
  Voltage Angle at Bus Bus2: -180.00 degrees
  Voltage Angle at Bus Bus1: 180.00 degrees


Optimal Solution:
Hour 1:
  Generator Gen1: Energized = No, Power Generated = 0.00 MW
  Generator Gen2: Energized = No, Power Generated = 0.00 MW
  Line Line2: Energized = Yes
  Line Line1: Energized = Yes
  Bus Bus2: Energized = Yes
  Bus Bus1: Energized = Yes
  Load Load1 Shedding: 100.00% of 100 MW
  Load Load2